In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

**XGBoost**

O resultado utilizando XGBoost ficou longe do aceitável, acabei usando primeiramente só para ter ideia do resultado. Acabei rodando algumas vezes e desistindo de utilizar pelo resultado, partindo para a LSTM.

In [ ]:
#from sklearn.model_selection import train_test_split

#df_train=pd.read_csv(r"/kaggle/input/ventilator-pressure-prediction/train.csv",encoding='latin-1')
#df_train.head()

#X = df_train.drop(['pressure','id'], axis='columns')
#y = df_train['pressure']

#Xtr, Xval, ytr, yval = train_test_split(X, y, test_size = 0.2, random_state=0)


In [ ]:
#df_train.head()

In [ ]:
#df_test=pd.read_csv(r"/kaggle/input/ventilator-pressure-prediction/test.csv",encoding='latin-1')

#test = df_test.drop('id',axis='columns')

In [ ]:
#types = ['id','breath_id', 'R', 'C', 'time_step', 'u_in', 'u_out', 'pressure']

#df_train.dtypes

In [ ]:
#for tp in types:
    #print(len(corpus_treino.loc[corpus_treino[tp] == ' ']))
    #print(len(corpus_treino.loc[corpus_treino[tp] == 0]))
    #corpus_treino[tp] = pd.to_numeric(corpus_treino[tp])

In [ ]:
#from sklearn.metrics import mean_squared_error
#import xgboost as xgb

#model = xgb.XGBRegressor(learning_rate=0.01, n_estimators=500, random_state=0)

#model.fit(Xtr, ytr)

In [ ]:
#predictions = model.predict(Xval)

In [ ]:
#from sklearn.metrics import explained_variance_score
#print(explained_variance_score((yval), (predictions)))

In [ ]:
#np.sqrt(mean_squared_error(yval,predictions))

**LSTM**

LSTM foi projetado para evitar o problema de dependência de longo prazo. O seu comportamento é lembrar de informações por longos períodos de tempo, como o nosso problema é de séries temporais esse modelo acaba sendo muito efetivo.

In [ ]:
import numpy as np
import pandas as pd

import optuna

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.optimizers.schedules import ExponentialDecay

from sklearn.metrics import mean_absolute_error as mae
from sklearn.preprocessing import RobustScaler, normalize
from sklearn.model_selection import train_test_split, GroupKFold, KFold

from IPython.display import display

Primeiramente é realizada a leitura dos arquivos teste e treino disponibilizados pela competição. Nota-se que o tamanho do arquivo de treino foi diminuido, isso foi feito porque o modelo demora muito para ser treinado utilizando o arquivo inteiro. Com o arquivo completo, o modelo demorava muito para rodar, com essa diminuição do tamanho do arquivo foi possível rodar o modelo em 3 a 4 horas. Com isso, foi possível realizar testes de uma quantidade maior de variações em um tempo menor. Não foram realizados tantos testes a ponto de obter o melhor resultado, foram realizados alguns testes melhorando o resultado através de algumas features. O arquivo original tem 6036000 linhas, com a diminuição ficaram 80000 linhas. Entende-se que isso pode afetar muito no aprendizado do modelo, mas nesse caso foi escolhido correr esse risco pela demora que é rodar com o arquivo completo. No começo foi utilizado KFolds para gerar modelos diferentes, mas acabei deixando de lado pelo tempo que estava levando para treinar, não afetava muito o desempenho final.

In [ ]:
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')

O arquivo de teste tinha 6036000 linhas.

In [ ]:
train.shape

E passou a ter 80000 linhas.

In [ ]:
train = train[:80000]

In [ ]:
train.shape

In [ ]:
train.head()

In [ ]:
import seaborn as sns
sns.lineplot(x='breath_id',y='pressure', data=train)

Os dados foram disponibilizados de maneira que não precisam de tratamento. O que precisava ser feito era adicionar algumas features nos dados para ver se o modelo obtinha um resultado melhor.

Para realizar as features foram utilizadas algumas funções:
1. A função groupby() que agrupa os objetos de mesmo valor, aplica uma função e combina os resultados;
2. A função cumsum() que a soma os elementos ao longo de um determinado eixo;
3. A função shift() que é imprescindível, porque é ela que desloca os dados da coluna, é deslocado para usar dados anteriores para prever o futuro, sem deslocar estariamos prevendo de maneira errada. Diferentes valores de deslocamento foram testados para avaliar seus impactos.

In [ ]:
def add_features(df):
    
    #Gerando um time geral com o time_step, u_in e breath_id, depois o breath_id é retirado
    df['area'] = df['time_step'] * df['u_in']
    df['area'] = df.groupby('breath_id')['area'].cumsum()
    df['u_in_cumsum'] = (df['u_in']).groupby(df['breath_id']).cumsum()
    
    #Agrupando valores com o mesmo breath_id, adicionando diferentes valores de deslocamento
    df['u_in_lag1'] = df.groupby('breath_id')['u_in'].shift(1)
    df['u_out_lag1'] = df.groupby('breath_id')['u_out'].shift(1)
    df['u_in_lag_back1'] = df.groupby('breath_id')['u_in'].shift(-1)
    df['u_out_lag_back1'] = df.groupby('breath_id')['u_out'].shift(-1)
    df['u_in_lag2'] = df.groupby('breath_id')['u_in'].shift(2)
    df['u_out_lag2'] = df.groupby('breath_id')['u_out'].shift(2)
    df['u_in_lag_back2'] = df.groupby('breath_id')['u_in'].shift(-2)
    df['u_out_lag_back2'] = df.groupby('breath_id')['u_out'].shift(-2)
    df['u_in_lag3'] = df.groupby('breath_id')['u_in'].shift(3)
    df['u_out_lag3'] = df.groupby('breath_id')['u_out'].shift(3)
    df['u_in_lag_back3'] = df.groupby('breath_id')['u_in'].shift(-3)
    df['u_out_lag_back3'] = df.groupby('breath_id')['u_out'].shift(-3)
    df['u_in_lag4'] = df.groupby('breath_id')['u_in'].shift(4) #5
    df['u_out_lag4'] = df.groupby('breath_id')['u_out'].shift(4)
    df['u_in_lag_back4'] = df.groupby('breath_id')['u_in'].shift(-4)
    df['u_out_lag_back4'] = df.groupby('breath_id')['u_out'].shift(-4)
    
    #Preencher vazios
    df = df.fillna(0)

    #Subtraindo valores com deslocamentos diferentes
    df['u_in_diff1'] = df['u_in'] - df['u_in_lag1']
    df['u_out_diff1'] = df['u_out'] - df['u_out_lag1']
    df['u_in_diff2'] = df['u_in'] - df['u_in_lag2']
    df['u_out_diff2'] = df['u_out'] - df['u_out_lag2']
    df['u_in_diff3'] = df['u_in'] - df['u_in_lag3']
    df['u_out_diff3'] = df['u_out'] - df['u_out_lag3']
    df['u_in_diff4'] = df['u_in'] - df['u_in_lag4']
    df['u_out_diff4'] = df['u_out'] - df['u_out_lag4']
    
    df['R'] = df['R'].astype(str)
    df['C'] = df['C'].astype(str)
    df['R__C'] = df["R"].astype(str) + '__' + df["C"].astype(str)
    df = pd.get_dummies(df)
    return df

train = add_features(train)
test = add_features(test)

In [ ]:
train.head()

In [ ]:
# Retirando do arquivo de treino os valores de predição do modelo e o breath_id que foi utilizado 
# para gerar outra variável representando o tempo geral
targets = train[['pressure']].to_numpy().reshape(-1, 80)
train.drop(['pressure', 'id', 'breath_id'], axis=1, inplace=True)
test = test.drop(['id', 'breath_id'], axis=1)

In [ ]:
train.shape

In [ ]:
# Fazendo uma padronização dos conjunto de dados, redimensionando os datasets para evitar também
# os outliers.
RS = RobustScaler()
train = RS.fit_transform(train)
test = RS.transform(test)

In [ ]:
#Reshape nos datasets para ficarem com as mesmas dimensões
train = train.reshape(-1, 80, train.shape[-1])
test = test.reshape(-1, 80, train.shape[-1])

In [ ]:
targets.shape

O modelo escolhido LSTM é uma rede neural reccorente LSTM (Long Short Term Memory), muito utilizada para fazer previsões com base de dados de série temporal.
Como hyperparâmetros foram selecionados: epoch e batch_size. O batch_size é um hiperparâmetro que define o número de amostras a serem processadas antes da atualização do modelo. O número de epoch é um hiperparâmetro que define o número de vezes que o algoritmo de aprendizado funcionará em todo o conjunto de dados de treinamento.

In [ ]:
EPOCH = 300
BATCH_SIZE = 1024

In [ ]:
tamanho_treino = int(len(train)*0.9) # Pegando 90% dos dados para treino
tamanho_teste = len(train)-tamanho_treino # Pegando o resto para teste

#Os valores de treino serão utilizados para treinar o modelo, e os valores de teste para testar 
#o modelo conforme ele for sendo treinado, é possível treinar e analisar ao mesmo tempo

No treinamento do modelo (.fit()) é passado o parâmetro validation_data(X_valid, y_valid), esses dados de validação não são usados para o ajuste da rede, eles são utilizados para verificar a evolução da função de custo, tanto para os dados de treino quanto para os dados de teste conforme a rede for sendo treinada, isso é printado conforme a rede for sendo treinada, val_loss é o valor da função de custo para seus dados de validação cruzada e o loss é o valor da função de custo para seus dados de treinamento. 

Como comentado, o arquivo de treino tinha sido dividido em NUM_FOLDS vezes, gerando diferentes modelos treinados e testados por diferentes partes do arquivo de treino, o que se faz é pegar o modelo com maior capacidade de generalização entre os modelos gerados. Como o tempo estava muito longo, acabei deixando de lado essa parte.

O modelo criado tem 1 camada de entrada, 1 camada de saída e entre esses 2 camadas há 5 camadas invisíveis de 1024, 512, 256, 128 e 128 unidades.

O código foi executado utilizando uma TPU para treinar o modelo mais rápido. Foi utilizado o acelerador TPU v3-8 disponibilizado pelo kaggle.

In [ ]:
#Valores para treinar o modelo
X_train = train[0:tamanho_treino]
X_valid = train[tamanho_treino:len(train)]

#Valores para verificar a evolução
y_train = targets[0:tamanho_treino]
y_valid = targets[tamanho_treino:len(targets)]

#Iniciar a TPU para o uso, deixando o tempo de treinamento muito menor
tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

#Inicializando a estratégia de distribuição
tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

with tpu_strategy.scope():
        
    model = keras.models.Sequential([
        keras.layers.Input(shape=train.shape[-2:]),
        keras.layers.Bidirectional(keras.layers.LSTM(1024, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True)),
        keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True)),
        keras.layers.Dense(128, activation='selu'),
        keras.layers.Dense(1),
    ])
    model.compile(optimizer="adam", loss="mae")

    model.fit(X_train, y_train, validation_data=(X_valid, y_valid), epochs=EPOCH, batch_size=BATCH_SIZE)

A função LOSS escolhida foi a **Mean Absolute Error Loss (MAE)**, ela é calculada como a média da diferença absoluta entre os valores reais e previstos. Os valores finais do modelo nessa última execução obtiveram o **loss: 0.2994** (para os dados de treino) e o **val_loss: 0.7055** (para os dados de teste). Acredito que ainda é possível melhorar adicionando mais features que impactem no resultado final e mexer mais nos hyperparâmetros.